# README
state_dict = {"sleep":0, "preseizure": 1}

In [6]:
import numpy as np
import os 
import pickle

## config setting 

In [4]:
# filter
filter_pre_1_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/filter/pre_1"
filter_pre_2_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/filter/pre_2"
filter_sleep_path_dir = "..//visualization_feature/raw_data_time_sequentially/sleep/BDP/filter"

# non-filter
non_filter_pre_1_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/no_filter/pre_1"
non_filter_pre_2_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/no_filter/pre_2"
non_filter_sleep_path_dir = "..//visualization_feature/raw_data_time_sequentially/sleep/BDP/no_filter"

生成滤波和非滤波切片的映射

In [5]:
fi_pre_1_names = os.listdir(filter_pre_1_path_dir)
fi_pre_2_names = os.listdir(filter_pre_2_path_dir)
fi_sleep_names = os.listdir(filter_sleep_path_dir)
# sorted by time 
fi_pre_1_names.sort()
fi_pre_2_names.sort()
fi_sleep_names.sort()
print("fi_pre_1:{}, fi_pre_2:{}, fi_sleep:{}".format(len(fi_pre_1_names), len(fi_pre_2_names), len(fi_sleep_names)))

nfi_pre_1_names = os.listdir(non_filter_pre_1_path_dir)
nfi_pre_2_names = os.listdir(non_filter_pre_2_path_dir)
nfi_sleep_names = os.listdir(non_filter_sleep_path_dir)
nfi_pre_1_names.sort()
nfi_pre_2_names.sort()
nfi_sleep_names.sort()
print("nfi_pre_1:{}, nfi_pre_2:{}, nfi_sleep:{}".format(len(nfi_pre_1_names), len(nfi_pre_2_names), len(nfi_sleep_names)))


fi_nfi_dict = {}
label = 1
for m, n in zip(fi_pre_1_names+fi_pre_2_names, nfi_pre_1_names+nfi_pre_2_names):
    fi_nfi_dict[m] = {"nfi_id":n, "label": label}
label = 0
for m ,n in zip(fi_sleep_names, nfi_sleep_names):
    fi_nfi_dict[m] = {"nfi_id":n, "label": label}

print(len(fi_nfi_dict))

fi_pre_1:309, fi_pre_2:1783, fi_sleep:7199
nfi_pre_1:309, nfi_pre_2:1783, nfi_sleep:7199
9291


In [8]:
print(fi_nfi_dict)

{'e076f2d4-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69ee6-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2d5-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69ee7-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2d6-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69ee8-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2d7-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69ee9-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2d8-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69eea-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2d9-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69eeb-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2da-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69eec-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2db-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69eed-2ac8-11ea-a4b0-e0d55e6ff654-0.npy', 'label': 1}, 'e076f2dc-2552-11ea-9699-e0d55e6ff654-0.npy': {'nfi_id': '54f69eee-2ac8

将映射表进行保存

保存格式：{fi_id:{"nfi_id":nfi_id, "label":0/1 }}

fi_id: 经过滤波切片的id，  nfi_if: 没有经过滤波的切片id

In [ ]:
sava_file = "./data/fi_nfi_id.pkl"
with open(sava_file, 'wb') as f:
    pickle.dump(fi_nfi_dict, f)
    print("映射表保存成功！")